# Exercise 4: Image Data Preparation

## Objective
Understand how images become numerical data and prepare them for a vision model.

We will:
- Capture images (robustly, handling cloud environments)
- Convert images to numerical arrays
- Resize and normalize images
- Apply simple data augmentation

This is the foundation of computer vision systems.

In [ ]:
!pip install -q opencv-python-headless matplotlib numpy

## Step 1: Capture an Image (The "Cloud-Safe" Way)

Standard `cv2.VideoCapture(0)` fails in cloud environments (Colab/Kaggle). We use a fallback mechanism.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_image_robust():
    # Try to open local webcam
    cap = cv2.VideoCapture(0)
    if cap.isOpened():
        ret, frame = cap.read()
        cap.release()
        if ret:
            print("Success: Captured image from local webcam.")
            return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Fallback: Generate a synthetic test pattern
    print("Warning: Webcam not found (common in cloud). Generating synthetic image.")
    height, width = 480, 640
    # Create a gradient image
    img = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(3):
        img[:, :, i] = np.linspace(0, 255, width)
    
    # Draw a circle so we can see augmentations later
    cv2.circle(img, (320, 240), 100, (255, 255, 0), -1)
    return img

frame_rgb = get_image_robust()

plt.imshow(frame_rgb)
plt.title("Input Image")
plt.axis("off")
plt.show()

print("Image shape:", frame_rgb.shape)

## Step 2: Resize Image

Neural networks require fixed input size (e.g., 224x224 for ResNet).

In [ ]:
target_size = (224, 224)
resized = cv2.resize(frame_rgb, target_size)

plt.imshow(resized)
plt.title(f"Resized to {target_size}")
plt.axis("off")
plt.show()

## Step 3: Normalize Pixels

Pixel values range from 0–255. We scale them to 0–1 for numerical stability in the neural network.

In [ ]:
normalized = resized.astype('float32') / 255.0

print(f"Min pixel value: {normalized.min():.2f}")
print(f"Max pixel value: {normalized.max():.2f}")
print(f"Data Type: {normalized.dtype}")

## Step 4: Data Augmentation

We apply simple transformations to increase dataset diversity. Note: We use the *original* (0-255) image for OpenCV functions, as some don't like floats.

In [ ]:
# Flip Horizontal
flipped = cv2.flip(resized, 1)

# Increase Brightness (Manual method to avoid overflow/underflow)
# We convert to int16 to prevent wrapping (255+1 = 0), then clip.
bright_temp = resized.astype(np.int16) + 50
bright = np.clip(bright_temp, 0, 255).astype(np.uint8)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(flipped)
plt.title("Flipped")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(bright)
plt.title("Brightness Increased")
plt.axis("off")
plt.show()